# Autor: Jéssyca Noronha ♥

Este projeto tem como objetivo comparar diferentes modelos de classificação de preços de celulares utilizando características como 'battery_power', 'clock_speed', 'fc', 'int_memory', 'n_cores', 'pc', 'ram' e 'talk_time'.


Etapas do Projeto:

    Carga dos Dados: Os dados são carregados a partir de um arquivo CSV (exemplo fictício) e selecionadas as colunas relevantes para a análise. O conjunto de dados inclui informações sobre atributos técnicos dos celulares e a classe de preço correspondente (price_range).

    Separação de Features e Target: As características (X) são extraídas, removendo a coluna de preços (price_range), que se torna a variável alvo (y).

    Separação Treino/Teste: O conjunto de dados é dividido em dados de treino e teste, utilizando uma proporção de 80% para treino e 20% para teste, garantindo que o modelo possa ser avaliado adequadamente.

    Normalização/Padronização: Pipelines são criados para normalizar os dados usando diferentes técnicas de escalonamento, como StandardScaler e MinMaxScaler, aplicados em combinação com vários classificadores: K-Nearest Neighbors (KNN), Decision Tree, Naive Bayes e Support Vector Machine (SVM).

    Otimização de Hiperparâmetros: Para cada modelo, uma busca em grade (GridSearchCV) é realizada para encontrar os melhores hiperparâmetros, usando validação cruzada para garantir a robustez dos resultados.

    Avaliação dos Modelos: Após o treinamento, cada modelo é avaliado com base em métricas de desempenho, como precisão, recall e F1-score, apresentando um relatório de classificação e a acurácia obtida em relação ao conjunto de teste.

    Exportação do Melhor Modelo: O modelo que apresentou o melhor desempenho (neste caso, o SVM) é salvo em um arquivo para utilização futura.

Este fluxo de trabalho fornece uma base sólida para a classificação de preços de celulares, permitindo que os usuários comparem e escolham o modelo mais adequado para suas necessidades.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline
import joblib

# 1. Carga dos dados (exemplo com dataset fictício)
df_1 = pd.read_csv('https://raw.githubusercontent.com/Jessycanoronha/mobilePriceClassification/main/api/dataset/train.csv')
df = df_1[['battery_power', 'clock_speed', 'fc', 'int_memory', 'n_cores', 'pc', 'ram', 'talk_time', 'price_range']];

# Separando as features e o target
X = df.drop('price_range', axis=1)
y = df['price_range']

# 2. Separação treino/teste (holdout)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Normalização/Padronização (usando Pipeline)
pipelines = {
    'knn': Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier())]),
    'tree': Pipeline([('scaler', StandardScaler()), ('tree', DecisionTreeClassifier())]),
    'nb': Pipeline([('scaler', MinMaxScaler()), ('nb', GaussianNB())]),
    'svm': Pipeline([('scaler', StandardScaler()), ('svm', SVC())])
}

# 4. Otimização de Hiperparâmetros (GridSearchCV para cada modelo)
param_grids = {
    'knn': {'knn__n_neighbors': [3, 5, 7]},
    'tree': {'tree__max_depth': [5, 10, 15]},
    'svm': {'svm__C': [0.1, 1, 10], 'svm__kernel': ['linear', 'rbf']}
}

best_models = {}
for name, pipeline in pipelines.items():
    grid = GridSearchCV(pipeline, param_grids.get(name, {}), cv=5)
    grid.fit(X_train, y_train)
    best_models[name] = grid.best_estimator_

# 5. Avaliação dos Modelos
for name, model in best_models.items():
    y_pred = model.predict(X_test)
    print(f"Resultados para {name}:")
    print(classification_report(y_test, y_pred))
    print(f"Acurácia: {accuracy_score(y_test, y_pred):.4f}")

# 6. Exportação do melhor modelo
melhor_modelo = best_models['svm']  # Exemplo de seleção do melhor modelo
joblib.dump(melhor_modelo, 'melhor_modelo.pkl')

Resultados para knn:
              precision    recall  f1-score   support

           0       0.82      0.87      0.84       105
           1       0.58      0.57      0.57        91
           2       0.51      0.59      0.55        92
           3       0.86      0.72      0.79       112

    accuracy                           0.69       400
   macro avg       0.69      0.69      0.69       400
weighted avg       0.71      0.69      0.70       400

Acurácia: 0.6950
Resultados para tree:
              precision    recall  f1-score   support

           0       0.93      0.90      0.91       105
           1       0.73      0.76      0.75        91
           2       0.62      0.75      0.68        92
           3       0.90      0.75      0.82       112

    accuracy                           0.79       400
   macro avg       0.80      0.79      0.79       400
weighted avg       0.81      0.79      0.79       400

Acurácia: 0.7900
Resultados para nb:
              precision    recall

['melhor_modelo.pkl']

In [ ]:
xt = {'battery_power': 1000,'clock_speed': 2.0,'fc': 10,'int_memory': 32,'n_cores': 8,'pc': 5,'ram': 2048, 'talk_time': 12}
d = pd.DataFrame(xt, index=[0])
y_pred = model.predict(d)
print(y_pred)

[1]


In [ ]:
filename = 'melhor_modelo.pkl'
with open(filename, 'rb') as file:
  m = joblib.load(file)

xt = {'battery_power': 4000,'clock_speed': 2.0,'fc': 10,'int_memory': 32,'n_cores': 8,'pc': 5,'ram': 2048, 'talk_time': 12}
input_data = pd.DataFrame(xt , index=[0])

y = m.predict(input_data)

print(y)

[3]
